# Enhancing Text Classification in Information Retrieval: A Comprehensive Approach with TF-IDF, Naive Bayes, Word Embeddings, LSA, and SVM


In [4]:
from nltk.tokenize import word_tokenize


from nltk.corpus import stopwords # a library to tokenize input texts

import nltk


nltk.download('punkt')
nltk.download('stopwords') # stopping word in English language

import string # using to remove punctuation

import numpy as np

from heapq import heappop, heappush, heapify

import copy

import matplotlib.pyplot as plt

import pandas as pd 

import string
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from gensim.models import Word2Vec, FastText
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

[nltk_data] Downloading package punkt to /Users/amirali/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amirali/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# NAIVE BAYES 

In [52]:

class searchEngineNB:
    def __init__(self ) -> None: # constructor of class
        self.avrLen =0
        self.postingList =[]
        self.posPostingList=[]
        self.negPostingList=[]
        self.files=[] 
        self.numberOfDocuments = 0
        self.numberOfPosDocuments = 0
        self.numberOfNegDocuments = 0
        self.tfIdfMatrix = []
        self.tfIdfMatrixPos = []
        self.tfIdfMatrixNeg = []
        self.totalWords = 0
        self.totalWordsPos = 0
        self.totalWordsNeg = 0
        self.naveBayes = {
            "pos":{},
            "neg":{}
        }

        self.data = []

        # Stopwords set to ignore common words in English
        self.stop = set(stopwords.words('english') + list(string.punctuation)) # all extra expression which should ignore

        # structure of postingList : list of {word : nameOfWord , docs :[list of {doc:nameOfDocument , indexes: indexes of the word if this document}]}


    # binary search to find a word in posting list
    def searchPostingList(self, word,PostingList):

        # Binary search to find a word in the posting list.

        # Parameters:
        # - word: The word to search for in the posting list.

        # Returns:
        # - Index of the word in the posting list.

        s= 0 
        e = len(PostingList)
        if e <=0 :
            return 0
        e-=1
        while (1):
            if (e-s < 2):
                if (PostingList[e]["word"] < word):
                    return e+1
                if (PostingList[e]["word"] == word):
                    return e
                if (PostingList[s]["word"] >= word):
                    return s

                return e
            mid = (s+e)/2
            mid = int(mid)
            if (word<PostingList[mid]["word"]):
                e=mid
            elif (word> PostingList[mid]["word"]):
                s = mid
            else :
                return mid
            
        # {word:str, indexes:list(int)}
    # binary search to find a word in each dictionary



    def addToPostingList(self, tokenizedText: list[str],docIndex:int , PostingList,mode = "main"  ): # add tokenized word in posting list 

        # Add tokenized words to the posting list.

        # Parameters:
        # - tokenizedText: List of tokenized words in a document.
        # - docIndex: Index of the document.
        # - title, author, B: Metadata of the document.

        # Returns:
        # None
        if mode == "main" :
            self.numberOfDocuments+=1
        elif mode == "pos" :
            self.numberOfPosDocuments+=1
        else :
            self.numberOfNegDocuments+=1
        for i in range(len(tokenizedText)):
            if mode == "main" :
                self.totalWords+=1
            elif mode == "pos" :
                self.totalWordsPos+=1
            else :
                self.totalWordsNeg+=1
            word = tokenizedText[i]
            index = self.searchPostingList(word,PostingList) # find index of word in posting list
            if (len(PostingList)>index): # check if index is not larger than posting list (if word is bigger that all words, search function returns len(postingList)+1)
                if (PostingList[index]["word"] == word): # check if index is the index of the word
                    if (PostingList[index]["docs"][-1]["doc"] == docIndex): # if we have already added the document index 
                        PostingList[index]["docs"][-1]["indexes"].append(i) # as we read tokens in order of their index, we need to add token in end of the list
                        PostingList[index]["docs"][-1]["tf"]+=1
                        PostingList[index]["cf"]+=1


                    else:
                        PostingList[index]["docs"].append({"doc":docIndex,"indexes":[i], "tf":1  }) # if we have not already added the document and dou to the fact that they are read in order of their index, we can easily add append new one in end of the list 
                        PostingList[index]["df"]+=1
                        PostingList[index]["cf"]+=1
                        

                else :
                    PostingList[index:index]= [({"word":word ,"df":1 ,"cf":1 , "docs":[{"doc":docIndex,"indexes":[i], "tf":1  }]})] # word is bigger that all other words => we can append it to end of the list
                                                    
            else :
                PostingList.append({"word":word ,"df":1 ,"cf":1 , "docs":[{"doc":docIndex,"indexes":[i], "tf":1 }]}) # we have not already added the word and dou to the fact that they are read in order of their index, we can easily add append new one in end of the list



    def input (self, dataFrame, limit = -1,printDetails = False): # input paths of inputs


        
        totalLen = 0
        data = dataFrame
        self.data = data
        self.numberOfDocuments = data.shape[0]
        for i in range(self.numberOfDocuments ) :
            if (i == limit ):break
            file = data.iloc[i]
            text = file["text"] # read the file
            # tokenize text and ignore stopping words using nltk library 
            tokenizedText = [word for word in word_tokenize(text.lower(),preserve_line=False) if word not in self.stop] 
            printDetails and print (f'document {i+1} tokenized : {tokenizedText}')
            totalLen+=len(tokenizedText)
            # print(tokenizedText)
            self.addToPostingList(tokenizedText , i+1 , self.postingList) # i indicates to index of document we are reading
            if file['rating'] >5:
                self.addToPostingList(tokenizedText , file["ID"]+1 , self.posPostingList,mode="pos") # i indicates to index of document we are reading
            else:
                self.addToPostingList(tokenizedText , file["ID"]+1 , self.negPostingList,mode="neg") # i indicates to index of document we are reading

        self.avrLen = totalLen/self.numberOfDocuments


    

    def trainNaveBayes (self):
        # this function trans naive byes to predict class of the queries.

        self.naveBayes = { # clear previous data
            "pos":{},
            "neg":{}
        }

        totalNumberOfTerms = len(self.postingList) # B
        self.negDiv = (self.totalWordsNeg+ totalNumberOfTerms) 
        self.posDiv = (self.totalWordsPos+ totalNumberOfTerms)

        for term in self.posPostingList: # find probabilities of positive class words
            word = term["word"]
            cf = term["cf"]
            self.naveBayes["pos"][word] = np.log(cf / self.posDiv)
             
        for term in self.negPostingList:# find probabilities of negative class words
            word = term["word"]
            cf = term["cf"]
            self.naveBayes["neg"][word] = np.log(cf / self.negDiv)

            


    def query (self, q ,printDetails=False ): # input the query and predict its class
        q = str(q)
        tokenizedQuery= [word for word in word_tokenize(q.lower(),preserve_line=False) if word not in self.stop] 
        PP = 0
        PN = 0
        TrainedDataPos = self.naveBayes["pos"]
        TrainedDataNeg = self.naveBayes["neg"]
        for term in tokenizedQuery:
            try:
                temp = TrainedDataPos[term]
                PP+=temp
                printDetails and print (f'word {term} exists in pos with value {temp}')
            except:
                PP+= np.log(1/self.posDiv)

            try:
                temp = TrainedDataNeg[term]
                PN+=temp
                printDetails and print (f'word {term} exists in neg with value {temp}')
            except:
                PN+= np.log(1/self.negDiv)
            # PP+= np.log(1/self.posDiv)
            # PN+= np.log(1/self.negDiv)
        printDetails and print(f'positive probability is {PP} and negative probability is {PN}')
        return PP>=PN 

    def build (self):

        # Build the index.

        # Parameters:
        # None

        # Returns:
        # None
        self.trainNaveBayes()
        self.makeIDF()
        self.makeTfIdfMatrix()

    def makeIDF (self):

        # Calculate IDF for terms in the posting list.

        # Parameters:
        # None

        # Returns:
        # None

        for item in self.postingList:
            item["idf"] = np.log(self.numberOfDocuments) - np.log((item["df"] or 1))

        for item in self.posPostingList:
            item["idf"] = np.log(self.numberOfPosDocuments) - np.log((item["df"] or 1))

        for item in self.negPostingList:
            item["idf"] = np.log(self.numberOfNegDocuments) - np.log((item["df"] or 1))


    def makeTfIdfMatrix(self): # create to matrix tfIdf and tdIdfPm

        #  Calculate TF-IDF matrices for documents.

        # Parameters:
        # None

        # Returns:
        # None

        self.tfIdfMatrix =  [[0]*self.numberOfDocuments for _ in range(len(self.postingList))]
        self.tfIdfMatrixPos =  [[0]*self.numberOfPosDocuments for _ in range(len(self.posPostingList))]
        self.tfIdfMatrixNeg =  [[0]*self.numberOfNegDocuments for _ in range(len(self.negPostingList))]
        for indx in range(len(self.postingList)):
            item = self.postingList[indx]
            idf = item["idf"]
            docs = item["docs"]
            for doc in docs:
                self.tfIdfMatrix[indx][doc["doc"]-1]=doc["tf"] * idf

        for indx in range(len(self.posPostingList)):
            item = self.posPostingList[indx]
            idf = item["idf"]
            docs = item["docs"]
            for doc in docs:
                self.tfIdfMatrixPos[indx][doc["doc"]-1]=doc["tf"] * idf

        for indx in range(len(self.negPostingList)):
            item = self.negPostingList[indx]
            idf = item["idf"]
            docs = item["docs"]
            for doc in docs:
                self.tfIdfMatrixNeg[indx][doc["doc"]-1]=doc["tf"] * idf
                


        self.tfIdfMatrix = np.array(self.tfIdfMatrix ,dtype=float)
        self.tfIdfMatrixPos = np.array(self.tfIdfMatrixPos ,dtype=float)
        self.tfIdfMatrixNeg = np.array(self.tfIdfMatrixNeg ,dtype=float)

        
        for doc in range(self.numberOfDocuments):
            norm =  np.linalg.norm(self.tfIdfMatrix[:,doc])



    def Test (self, dataFrame, limit = -1,printDetails = False): 
        
        TP = FP = FN= TN = 0
        data = dataFrame
        numberOfDocuments = data.shape[0]
        for i in range(numberOfDocuments ) :
            if (i == limit ):break
            file = data.iloc[i]
            text = file["text"] 
            rating = file["rating"]
            result = self.query(text)
            if (rating > 5):
                if result:
                    TP +=1
                else :
                    FN +=1
            else :
                if result:
                    FP +=1
                else :
                    TN +=1
        
        return {
            "TP":TP,
            "TN":TN,
            "FP":FP,
            "FN":FN,

        }

    





# Word Embeddings, LSA, and SVM Part

In [97]:

class searchEngineLSA_SVM:
    def __init__(self ) -> None: # constructor of class
        self.avrLen =0
        self.postingList =[]
        self.posPostingList=[]
        self.negPostingList=[]
        self.files=[] 
        self.numberOfDocuments = 0
        self.numberOfPosDocuments = 0
        self.numberOfNegDocuments = 0
        self.tfIdfMatrix = []
        self.tfIdfMatrixPos = []
        self.tfIdfMatrixNeg = []
        self.totalWords = 0
        self.totalWordsPos = 0
        self.totalWordsNeg = 0
        self.xTrain=[]
        self.xTest=[]
        self.yTest=[]
        self.yTest=[]
        self.naveBayes = {
            "pos":{},
            "neg":{}
        }

        self.data = []

        # Stopwords set to ignore common words in English
        self.stop = set(stopwords.words('english') + list(string.punctuation)) # all extra expression which should ignore

        # structure of postingList : list of {word : nameOfWord , docs :[list of {doc:nameOfDocument , indexes: indexes of the word if this document}]}


    # binary search to find a word in posting list
    def searchPostingList(self, word,PostingList):

        # Binary search to find a word in the posting list.

        # Parameters:
        # - word: The word to search for in the posting list.

        # Returns:
        # - Index of the word in the posting list.

        s= 0 
        e = len(PostingList)
        if e <=0 :
            return 0
        e-=1
        while (1):
            if (e-s < 2):
                if (PostingList[e]["word"] < word):
                    return e+1
                if (PostingList[e]["word"] == word):
                    return e
                if (PostingList[s]["word"] >= word):
                    return s

                return e
            mid = (s+e)/2
            mid = int(mid)
            if (word<PostingList[mid]["word"]):
                e=mid
            elif (word> PostingList[mid]["word"]):
                s = mid
            else :
                return mid
            
        # {word:str, indexes:list(int)}
    # binary search to find a word in each dictionary



    def addToPostingList(self, tokenizedText: list[str],docIndex:int , PostingList,mode = "main"  ): # add tokenized word in posting list 

        # Add tokenized words to the posting list.

        # Parameters:
        # - tokenizedText: List of tokenized words in a document.
        # - docIndex: Index of the document.
        # - title, author, B: Metadata of the document.

        # Returns:
        # None
        if mode == "main" :
            self.numberOfDocuments+=1
        elif mode == "pos" :
            self.numberOfPosDocuments+=1
        else :
            self.numberOfNegDocuments+=1
        for i in range(len(tokenizedText)):
            if mode == "main" :
                self.totalWords+=1
            elif mode == "pos" :
                self.totalWordsPos+=1
            else :
                self.totalWordsNeg+=1
            word = tokenizedText[i]
            index = self.searchPostingList(word,PostingList) # find index of word in posting list
            if (len(PostingList)>index): # check if index is not larger than posting list (if word is bigger that all words, search function returns len(postingList)+1)
                if (PostingList[index]["word"] == word): # check if index is the index of the word
                    if (PostingList[index]["docs"][-1]["doc"] == docIndex): # if we have already added the document index 
                        PostingList[index]["docs"][-1]["indexes"].append(i) # as we read tokens in order of their index, we need to add token in end of the list
                        PostingList[index]["docs"][-1]["tf"]+=1
                        PostingList[index]["cf"]+=1


                    else:
                        PostingList[index]["docs"].append({"doc":docIndex,"indexes":[i], "tf":1  }) # if we have not already added the document and dou to the fact that they are read in order of their index, we can easily add append new one in end of the list 
                        PostingList[index]["df"]+=1
                        PostingList[index]["cf"]+=1
                        

                else :
                    PostingList[index:index]= [({"word":word ,"df":1 ,"cf":1 , "docs":[{"doc":docIndex,"indexes":[i], "tf":1  }]})] # word is bigger that all other words => we can append it to end of the list
                                                    
            else :
                PostingList.append({"word":word ,"df":1 ,"cf":1 , "docs":[{"doc":docIndex,"indexes":[i], "tf":1 }]}) # we have not already added the word and dou to the fact that they are read in order of their index, we can easily add append new one in end of the list



    def input (self, dataFrameTrain , dataFrameTest,buildPostingList = False, limit = -1,printDetails = False): # input paths of inputs

        # Input method to read documents from specified file CSV.

        # Parameters:
        # - dataFrame: List of file paths.
        # - limit: Limit the number of documents to read (default: -1 means read all).

        # Returns:
        # None

        totalLen = 0
        data = dataFrameTrain.iloc[:limit]
        self.datasetTest = dataFrameTest.iloc[:limit]
        self.dataset = data

        self.xTrain = dataFrameTrain["text"].to_list()[:limit]
        self.xTest = dataFrameTest["text"].to_list()[:limit]
        
        self.yTrain = ["pos" if i>5 else "neg"  for i in dataFrameTrain["rating"].to_list()[:limit]]
        self.yTest = ["pos" if i>5 else "neg"  for i in dataFrameTest["rating"].to_list()[:limit]]

        if (buildPostingList):
            self.numberOfDocuments = data.shape[0]
            for i in range(self.numberOfDocuments ) :
                if (i == limit ):break
                file = data.iloc[i]
                text = file["text"] # read the file
                # tokenize text and ignore stopping words using nltk library 
                tokenizedText = [word for word in word_tokenize(text.lower(),preserve_line=False) if word not in self.stop] 
                printDetails and print (f'document {i+1} tokenized : {tokenizedText}')
                totalLen+=len(tokenizedText)
                # print(tokenizedText)
                self.addToPostingList(tokenizedText , i+1 , self.postingList) # i indicates to index of document we are reading
                if file['rating'] >5:
                    self.addToPostingList(tokenizedText , file["ID"]+1 , self.posPostingList,mode="pos") # i indicates to index of document we are reading
                else:
                    self.addToPostingList(tokenizedText , file["ID"]+1 , self.negPostingList,mode="neg") # i indicates to index of document we are reading

            self.avrLen = totalLen/self.numberOfDocuments


    


    def word2vecModel(self,  vector_size=100, window=10, min_count=5, workers=4, sg=0):
        model = Word2Vec(self.xTrain, vector_size=vector_size, window=window, min_count=min_count, workers=workers, sg=sg)
        return model.wv

    def gloveModel(self, glove_file, word2vec_output_file):
        glove2word2vec(glove_file, word2vec_output_file)
        model = KeyedVectors.load_word2vec_format(word2vec_output_file)
        return model

    
    def fastTextModel(self,   vector_size=100, window=5, min_count=5, workers=4, sg=1):
        model = FastText(self.xTrain, vector_size=vector_size, window=window, min_count=min_count, workers=workers, sg=sg)
        return model.wv

    def makeDocumentEmbedding(self, doc, wVector):

        words = [word for word in doc if word in wVector]
        if not words:
            return np.zeros(wVector.vector_size)
        return np.mean(wVector[words], axis=0)
    
    def makeWordEmbeddings(self, wVector,mode="train"):
        data =  self.xTrain if mode == "train" else  self.xTest
        return [self.makeDocumentEmbedding(doc, wVector) for doc in data]
    
    def lsaing(self, dataTrain, dataTest, n_components=100):
        lsa = TruncatedSVD(n_components=n_components, random_state=42)
        xTrainLSA = lsa.fit_transform(dataTrain)
        xTestLsa = lsa.transform(dataTest)
        return xTrainLSA, xTestLsa

    def trainSvmClassifier(self,dataTrain, C=1.0, kernel='linear', gamma='auto'): # dataTrain can be embedding of terms
        svmClassifier = SVC(C=C, kernel=kernel, gamma=gamma)
        svmClassifier.fit(dataTrain, self.yTrain)
        return svmClassifier



    def Test (self,classifier,dataTest): 
        TP = TN = FP = FN = 0

        yPredicted = classifier.predict(dataTest)
        for i in range(len(yPredicted)):
            mainLabel = self.yTest[i] == "pos"
            predictLabel = yPredicted[i] == "pos"
            if (mainLabel ):
                if predictLabel:
                    TP +=1
                else :
                    FN +=1
            else :
                if predictLabel:
                    FP +=1
                else :
                    TN +=1
        
        return {
            "TP":TP,
            "TN":TN,
            "FP":FP,
            "FN":FN,

        }


    def train(self,mode="word2vecModel"):
        modeDic={
            "word2vecModel":self.word2vecModel,
            "gloveModel":self.gloveModel,
            "fastTextModel":self.fastTextModel,
        }
        trainer = modeDic[mode]
        wVector = trainer()

        # Create Word Embeddings
        xTrainWordEmb = self.makeWordEmbeddings( wVector)
        xTestWordEmb = self.makeWordEmbeddings( wVector,mode="test")


        # Apply LSA
        xTrainLSA, xTestLsa = self.lsaing(xTrainWordEmb, xTestWordEmb)

        # Train SVM Classifier
        svmClassifier = self.trainSvmClassifier(xTrainLSA)

        # Evaluate the Classifier
        return self.Test(svmClassifier,xTrainLSA)




    def build (self):

        # Build the index.

        # Parameters:
        # None

        # Returns:
        # None
        self.trainNaveBayes()
        self.makeIDF()
        self.makeTfIdfMatrix()

    def makeIDF (self):

        # Calculate IDF for terms in the posting list.

        # Parameters:
        # None

        # Returns:
        # None

        for item in self.postingList:
            item["idf"] = np.log(self.numberOfDocuments) - np.log((item["df"] or 1))

        for item in self.posPostingList:
            item["idf"] = np.log(self.numberOfPosDocuments) - np.log((item["df"] or 1))

        for item in self.negPostingList:
            item["idf"] = np.log(self.numberOfNegDocuments) - np.log((item["df"] or 1))


    def makeTfIdfMatrix(self): # create to matrix tfIdf and tdIdfPm

        #  Calculate TF-IDF matrices for documents.

        # Parameters:
        # None

        # Returns:
        # None

        self.tfIdfMatrix =  [[0]*self.numberOfDocuments for _ in range(len(self.postingList))]
        self.tfIdfMatrixPos =  [[0]*self.numberOfPosDocuments for _ in range(len(self.posPostingList))]
        self.tfIdfMatrixNeg =  [[0]*self.numberOfNegDocuments for _ in range(len(self.negPostingList))]
        for indx in range(len(self.postingList)):
            item = self.postingList[indx]
            idf = item["idf"]
            docs = item["docs"]
            for doc in docs:
                self.tfIdfMatrix[indx][doc["doc"]-1]=doc["tf"] * idf

        for indx in range(len(self.posPostingList)):
            item = self.posPostingList[indx]
            idf = item["idf"]
            docs = item["docs"]
            for doc in docs:
                self.tfIdfMatrixPos[indx][doc["doc"]-1]=doc["tf"] * idf

        for indx in range(len(self.negPostingList)):
            item = self.negPostingList[indx]
            idf = item["idf"]
            docs = item["docs"]
            for doc in docs:
                self.tfIdfMatrixNeg[indx][doc["doc"]-1]=doc["tf"] * idf
                


        self.tfIdfMatrix = np.array(self.tfIdfMatrix ,dtype=float)
        self.tfIdfMatrixPos = np.array(self.tfIdfMatrixPos ,dtype=float)
        self.tfIdfMatrixNeg = np.array(self.tfIdfMatrixNeg ,dtype=float)

        
        for doc in range(self.numberOfDocuments):
            norm =  np.linalg.norm(self.tfIdfMatrix[:,doc])









In [86]:

def printResults(result , printDetails=False):
    TP,TN,FP,FN = result["TP"],result["TN"],result["FP"],result["FN"]
    print ("                  Positive                 Negative           ")
    print ("       -------------------------------------------------------")
    print ("       |                         |                           |")
    print ("       |                         |                           |")
    print (" True  |      {0:8d}           |        {1:8d}           |".format(result["TP"], result["FP"]))
    print ("       |                         |                           |")
    print ("       |                         |                           |")
    print ("       -------------------------------------------------------")
    print ("       |                         |                           |")
    print ("       |                         |                           |")
    print (" False |      {0:8d}           |        {1:8d}           |".format(result["TN"], result["FN"]))
    print ("       |                         |                           |")
    print ("       |                         |                           |")
    print ("       -------------------------------------------------------")

    accuracy = (TP + TN) / (TP+TN+FN+FP)
    printDetails and print (f'accuracy : {accuracy}')
    print ()

    recall = (TP ) / (TP+FN)
    printDetails and print (f'recall : {recall}')
    print ()

    precision = (TP) / (TP+FP)
    printDetails and print (f'precision : {precision}')
    print ()

    F1 = (2*precision) / (precision + recall)
    printDetails and print (f'F1 : {F1}')
    print ()

    if printDetails :
        print (f'TP : {TP}')
        print (f'TN : {TN}')
        print (f'FP : {FP}')
        print (f'FN : {FN}')

    return {
        "accuracy" : accuracy,
        "recall" : recall,
        "precision" : precision,
        "F1" : F1,
    }

In [67]:
directory='dataSets'

# 1. Document Preprocessing:

In [14]:



dfPosTrain = pd.read_csv(f'{directory}/train_pos.csv').sort_values("ID")
dfNegTrain = pd.read_csv(f'{directory}/train_neg.csv').sort_values("ID")

dfPosTest = pd.read_csv(f'{directory}/test_pos.csv').sort_values("ID")
dfNegTest = pd.read_csv(f'{directory}/test_neg.csv').sort_values("ID")



In [ ]:
dfPosTrain.head()

,Unnamed: 0,ID,text,rating
0,0,0,Bromwell High is a cartoon comedy. It ran at t...,9
3611,3611,1,"If you like adult comedy cartoons, like South ...",7
4722,4722,2,Bromwell High is nothing short of brilliant. E...,9
5833,5833,3,"""All the world's a stage and its people actors...",10
6944,6944,4,FUTZ is the only show preserved from the exper...,8


In [ ]:
dfNegTrain.head()


,Unnamed: 0,ID,text,rating
0,0,0,Story of a man who has unnatural feelings for ...,3
3611,3611,1,Robert DeNiro plays the most unbelievably inte...,1
4722,4722,2,"I saw the capsule comment said ""great acting.""...",1
5833,5833,3,If I had not read Pat Barker's 'Union Street' ...,4
6944,6944,4,This fanciful horror flick has Vincent Price p...,4


In [ ]:
dfPosTest.head()


,Unnamed: 0,ID,text,rating
0,0,0,I went and saw this movie last night after bei...,10
3611,3611,1,My boyfriend and I went to watch The Guardian....,10
4722,4722,2,My yardstick for measuring a movie's watch-abi...,7
5833,5833,3,How many movies are there that you can think o...,7
6944,6944,4,This movie was sadly under-promoted but proved...,10


In [ ]:
dfNegTest.head()


,Unnamed: 0,ID,text,rating
0,0,0,Once again Mr. Costner has dragged out a movie...,2
3611,3611,1,This is a pale imitation of 'Officer and a Gen...,3
4722,4722,2,"It seems ever since 1982, about every two or t...",3
5833,5833,3,"Wow, another Kevin Costner hero movie. Postman...",4
6944,6944,4,"Alas, another Costner movie that was an hour t...",4


In [15]:
dfWholeTrain = pd.concat(
    (dfPosTrain,dfNegTrain),
    # axis=0,
    join="inner",
    ignore_index=True,
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    copy=True,
)

dfWholeTest = pd.concat(
    (dfPosTest,dfNegTest),
    # axis=0,
    join="inner",
    ignore_index=True,
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    copy=True,
)

In [16]:
dfWholeTrain

,Unnamed: 0,ID,text,rating
0,0,0,Bromwell High is a cartoon comedy. It ran at t...,9
1,3611,1,"If you like adult comedy cartoons, like South ...",7
2,4722,2,Bromwell High is nothing short of brilliant. E...,9
3,5833,3,"""All the world's a stage and its people actors...",10
4,6944,4,FUTZ is the only show preserved from the exper...,8
...,...,...,...,...
24995,2771,12495,"OK, I love bad horror. I especially love horro...",1
24996,2772,12496,To be brutally honest... I LOVED watching Seve...,1
24997,2773,12497,I'm sure that the folks on the Texas/Louisiana...,4
24998,2774,12498,This film has the kernel of a really good stor...,2


In [17]:
dfWholeTest

,Unnamed: 0,ID,text,rating
0,0,0,I went and saw this movie last night after bei...,10
1,3611,1,My boyfriend and I went to watch The Guardian....,10
2,4722,2,My yardstick for measuring a movie's watch-abi...,7
3,5833,3,How many movies are there that you can think o...,7
4,6944,4,This movie was sadly under-promoted but proved...,10
...,...,...,...,...
24995,2771,12495,CyberTracker is set in Los Angeles sometime in...,3
24996,2772,12496,Eric Phillips (Don Wilson) is a secret service...,3
24997,2773,12497,Plot Synopsis: Los Angeles in the future. Crim...,4
24998,2774,12498,"Oh, dear! This has to be one of the worst film...",1


In [18]:
SE = searchEngineNB()
# SE.input(dfWholeTrain.iloc[:10]  , printDetails=True)
SE.input(dfWholeTrain  , printDetails=False)

NameError: name 'searchEngineNB' is not defined

# 2. TF-IDF Vectorization:

In [ ]:
SE.makeIDF()
SE.makeTfIdfMatrix()

# 3. Naive Bayes Classification:

In [1]:

SE.trainNaveBayes()
SE.naveBayes

NameError: name 'SE' is not defined

In [252]:
SE.query(dfPosTest.iloc[1]["text"],printDetails=True )


True

In [253]:
SE.query(dfNegTest.iloc[1]["text"],printDetails=True )


False

In [241]:
result = SE.Test(dfWholeTest)
result

{'TP': 9630, 'TN': 10999, 'FP': 1501, 'FN': 2870}

In [245]:
TP,TN,FP,FN = result["TP"],result["TN"],result["FP"],result["FN"]


# NaiveBayes Result

                  Positive                 Negative           
       -------------------------------------------------------
       |                         |                           |
       |                         |                           |
 True  |          9630           |            1501           |
       |                         |                           |
       |                         |                           |
       -------------------------------------------------------
       |                         |                           |
       |                         |                           |
 False |         10999           |            2870           |
       |                         |                           |
       |                         |                           |
       -------------------------------------------------------


# Review NaiveBayes Results :

## accuracy

In [248]:
accuracy = (TP + TN) / (TP+TN+FN+FP)
print (f'accuracy : {accuracy}')

accuracy : 0.82516


## Recall

In [249]:
recall = (TP ) / (TP+FN)
print (f'recall : {recall}')

recall : 0.7704


## Precision

In [250]:
precision = (TP) / (TP+FP)
print (f'precision : {precision}')

precision : 0.8651513790315336


## F1

In [251]:
F1 = (2*precision) / (precision + recall)
print (f'F1 : {F1}')

F1 : 1.0579323769624644


In [98]:
SELSA = searchEngineLSA_SVM()

In [99]:
SELSA.input(dfWholeTrain,  dfWholeTest )
# rang= 10
# SampleDataTrain  = pd.concat(
#     (dfWholeTrain.iloc[:rang],dfNegTrain.iloc[:rang]),
#     # axis=0,
#     join="inner",
#     ignore_index=True,
#     keys=None,
#     levels=None,
#     names=None,
#     verify_integrity=False,
#     copy=True,
# )
# SampleDataTest  = pd.concat(
#     (dfWholeTest.iloc[:rang],dfNegTest.iloc[:rang]),
#     # axis=0,
#     join="inner",
#     ignore_index=True,
#     keys=None,
#     levels=None,
#     names=None,
#     verify_integrity=False,
#     copy=True,
# )
# SELSA.input(SampleDataTrain , SampleDataTest, printDetails=True)
# SampleDataTrain
# SampleDataTest


In [102]:
    # modeDic={
    #     "word2vecModel":self.word2vecModel,
    #     "gloveModel":self.gloveModel,
    #     "fastTextModel":self.fastTextModel,
    # }

In [103]:
result = SELSA.train(mode="word2vecModel")

In [101]:
result

{'TP': 8398, 'TN': 7822, 'FP': 4677, 'FN': 4102}

In [95]:
printResults(result , printDetails=True)

                  Positive                 Negative           
       -------------------------------------------------------
       |                         |                           |
       |                         |                           |
 True  |          8398           |            4677           |
       |                         |                           |
       |                         |                           |
       -------------------------------------------------------
       |                         |                           |
       |                         |                           |
 False |          7822           |            4102           |
       |                         |                           |
       |                         |                           |
       -------------------------------------------------------
accuracy : 0.6488259530381215

recall : 0.67184

precision : 0.6422944550669216

F1 : 0.9775171065493646

TP : 8398
TN

{'accuracy': 0.6488259530381215,
 'recall': 0.67184,
 'precision': 0.6422944550669216,
 'F1': 0.9775171065493646}